# Data Preprocessing

## Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
from matplotlib import cm
import plotly.graph_objs as go
import seaborn as sns

import cv2
import csv
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, f_regression, mutual_info_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

from skimage.feature import local_binary_pattern

from category_encoders import TargetEncoder,  CountEncoder

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from textblob import TextBlob
import re

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')
stop_words = set(stopwords.words('english'))

from datetime import datetime
from tqdm import tqdm
from scipy import stats

import requests
import os
import shutil

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from textblob import TextBlob
import re

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from datetime import datetime
from tqdm import tqdm
from scipy import stats

import requests
import os
import shutil

import warnings

warnings.filterwarnings('ignore')

In [ ]:
# Download nltk packages if not already downloaded
# nltk.download()

## Load the dataset

In [ ]:
dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y')

df_origin = pd.read_csv('games-regression-dataset.csv', 
                        parse_dates=['Original Release Date' , 'Current Version Release Date'], 
                        date_parser=dateparse)

## Data Gathering

### Download the icons

In [ ]:
# Convert to string
df_origin['Icon URL'] = df_origin['Icon URL'].astype(str)

# Download the images
def download_image(url, filename):
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)


# Create a folder to store the images
if not os.path.exists('icons'):
    os.makedirs('icons')

# Download the images if they don't exist
for i, row in tqdm(df_origin.iterrows(), total=df_origin.shape[0]):
    if not os.path.exists('icons/' + str(i) + '.png'):
        download_image(row['Icon URL'], 'icons/' + str(i) + '.png')
        
# Replace the URL with the icon filename which is the index of the row
df_origin['Icon URL'] = df_origin.apply(lambda row : f'icons/{row.name}.png', axis=1)

### Download the reviews

In [ ]:
def web_scrapping():
    data = pd.DataFrame(columns = ["ID","Reviews"])
    # Read CSV file
    # with open('games-regression-dataset.csv', newline='') as csvfile:
    with open('games-regression-dataset.csv', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header row
        for row in reader:
            url = row[0]  # URL is in first column
            filename = 'Reviews/'+os.path.basename(url)  # Extract filename from URL
            url +=  "?see-all=reviews"
            response = requests.get(url)
            if response.status_code == 200:  # Check if request was successful
                soup = BeautifulSoup(response.text, 'html.parser')
                blocks = soup.findAll("blockquote")
                review_list = []
                for blockquote in blocks:
                    review = blockquote.find('p').text
                    review_list.append(review)
                if len(review_list)!=0:
                    filename = re.sub(r'[^\d]+', '', filename)
                    new_row = {'ID': filename,"Reviews": review_list}
                    data = data._append(new_row, ignore_index=True)
                    
    data.to_csv('Reviews.csv', index=False)

In [ ]:
def reviews_splitting(data):

    for i in range (len(data)):
        data.at[i, 'Reviews'] = data.at[i, "Reviews"].split("',")
        data.at[i,"ID"] =data.at[i,"ID"]
        
    data = data.explode('Reviews')
    return data

In [ ]:
def reviews_cleaning(data):
  # Convert text to lowercase
  data['Reviews'] = data['Reviews'].apply(lambda x: str(x).lower())

  # Replace newline characters with an empty string
  data['Reviews'] = data['Reviews'].apply(lambda x: re.sub(r'\\n', ' ', x))

  # Remove black squares
  data['Reviews'] = data['Reviews'].apply(lambda x: re.sub(r'\\u25a0', '', x))

  # Remove special characters and punctuations
  data['Reviews'] = data['Reviews'].apply(lambda x: re.sub(r'[^\w\s]+', '', x))

  # Remove numbers
  data['Reviews'] = data['Reviews'].apply(lambda x: " ".join([word for word in x.split() if not any(char.isdigit() for char in word)]))

  # Remove extra whitespaces
  data['Reviews'] = data['Reviews'].apply(lambda x: re.sub(r'\s+', ' ', x))

  # Remove stop words
  data['Reviews'] = data['Reviews'].apply(lambda x: " ".join([word for word in x.lower().split() if word not in stop_words]))

  # Remove empty strings
  data = data[data['Reviews'].apply(lambda x: len(x)>0)]
  
  # Group by ID
  data = data.groupby('ID')['Reviews'].apply(list).reset_index()
  
  return data

In [ ]:
# web_scrapping()

df_reviews = pd.read_csv('Reviews.csv')

df_reviews = reviews_splitting(df_reviews)

df_reviews = reviews_cleaning(df_reviews)

## Merge The Sentiment with the original dataset
df_origin = df_origin.merge(df_reviews, on='ID', how='left')

## drop the rows with no reviews
df_origin = df_origin.dropna(subset=['Reviews'])

In [ ]:
df_origin.shape

## Reviews Preprocessing

In [ ]:
def reviews_preprocess(data, test=False):
  
  # Apply sentiment_analysis
  
  if test == False:
    sia_reviews = SentimentIntensityAnalyzer()
    pickle.dump(sia_reviews, open('encoders/sia_reviews.pkl', 'wb'))
  else:
    sia_reviews = pickle.load(open('encoders/sia_reviews.pkl', 'rb'))
  
  data['Reviews'] = data['Reviews'].apply(lambda x: [sia_reviews.polarity_scores(review)['compound'] for review in x])
  
  # Get the lowest, highest and average Reviews
  data['lowest_review'] = data['Reviews'].apply(lambda x: min(x) if len(x) > 0 else 0)
  data['highest_review'] = data['Reviews'].apply(lambda x: max(x) if len(x) > 0 else 0)
  data['average_review'] = data['Reviews'].apply(lambda x: np.mean(x) if len(x) > 0 else 0)
  
  ## Drop nulls of sentiment
  # data = data.dropna(subset=['lowest_review','highest_review','average_review'])
  return data

## Data Exploration

### General information

In [ ]:
# df.head()

In [ ]:
# df.info()

In [ ]:
# df.isnull().sum()

### Genres

In [ ]:
def genres_analysis(_df):
    _df['Genres'] = _df['Genres'].astype(str)
    _df['Genres'] = _df['Genres'].str.strip('[]').str.replace("'", "").str.split(", ")

    genre_counts = _df.explode('Genres').groupby('Genres').size().sort_values(ascending=False)
    print(genre_counts)


### Developer

In [ ]:
def dev_analysis(_df):

    print(_df['Developer'].value_counts())
    
    # print the number developers with more than 1 game
    print(len(_df['Developer'].value_counts()[_df['Developer'].value_counts() > 1]))

    print(_df['Developer'].unique().size)

### Dates

In [ ]:
def date_analysis(_df):
    # Plot the distribution of the date columns

    fig, ax = plt.subplots(5, 2, figsize=(20, 20))

    # df = date_preprocessing(df)

    # game_age distribution
    sns.histplot(_df['game_age'], ax=ax[0, 0])
    sns.boxplot(_df['game_age'], ax=ax[0, 1], orient='h')

    # last_update distribution
    sns.histplot(_df['last_update'], ax=ax[1, 0])
    sns.boxplot(_df['last_update'], ax=ax[1, 1], orient='h')

    # Original Release Date distribution
    sns.histplot(_df['Original Release Date'], ax=ax[2, 0])
    sns.boxplot(_df['Original Release Date'], ax=ax[2, 1], orient='h')

    # Current Version Release Date distribution
    sns.histplot(_df['Current Version Release Date'], ax=ax[3, 0])
    sns.boxplot(_df['Current Version Release Date'], ax=ax[3, 1], orient='h')

    # maintaning_period distribution
    sns.histplot(_df['maintaning_period'], ax=ax[4, 0])
    sns.boxplot(_df['maintaning_period'], ax=ax[4, 1], orient='h')

    plt.show()

# Date outliers are legitimate data points that are worth keeping, they are not errors nor anomalies

### Languages

In [ ]:
def lang_analysis(_df):
    _df['Languages'] = _df['Languages'].astype(str)
    _df['Languages'] = _df['Languages'].str.strip('[]').str.replace("'", "").str.split(", ")

    langs_counts = _df.explode('Languages').groupby('Languages').size().sort_values(ascending=False)
    print(langs_counts[1:30])


In [ ]:
# df.hist(figsize=(15, 15))

## Dates preprocessing

In [ ]:
def date_preprocess(_df, test=False, df_old=None):
    # Convert the datetime to ordinal
    _df['Original Release Date'] = _df['Original Release Date'].apply(lambda x: x.toordinal())
    _df['Current Version Release Date'] = _df['Current Version Release Date'].apply(lambda x: x.toordinal())
    
    if test == False:
        # Fill the missing values with the median
        _df['Original Release Date'] = _df['Original Release Date'].fillna(_df['Original Release Date'].median())
        _df['Current Version Release Date'] = _df['Current Version Release Date'].fillna(_df['Current Version Release Date'].median())
    else:
        # Fill the missing values with the median of the old dataset
        _df['Original Release Date'] = _df['Original Release Date'].fillna(df_old['Original Release Date'].median())
        _df['Current Version Release Date'] = _df['Current Version Release Date'].fillna(df_old['Current Version Release Date'].median())
    
    # Create a new column with the age of the game
    _df['game_age'] = datetime.now().toordinal() - _df['Original Release Date']

    # Create a new column with the time since the last update
    _df['last_update'] = datetime.now().toordinal() - _df['Current Version Release Date']
    
    # Create a new column with the maintaning period
    _df['maintaning_period'] = _df['game_age'] - _df['last_update']

    return _df

## Dates preprocessing

### Target encoding approach

In [ ]:
def dev_preprocess_target_enc(_df, test=False):
    """Preprocesses and encodes the 'Developer' column using target encoding.

    Args:
        df (DataFrame): DataFrame (either train or test, depending on the value of test)
        test (bool, optional): Boolean flag indicating whether the data is for testing (True) or training (False). Defaults to False.

    Returns:
        DataFrame: the preprocessed DataFrame with the new 'dev_avg' column
    """

    # Convert Developer column to string
    _df['Developer'] = _df['Developer'].astype(str)
    _df['Developer'] = _df['Developer'].str.replace("'", "").str.strip('[]')  
      
    # Replace Developers with less than 2 games with 'Other'
    if not test:
        dev_counts = _df['Developer'].value_counts()
        other = dev_counts[dev_counts < 2].index
        _df['Developer'] = _df['Developer'].replace(other, 'Other')

    # Perform target encoding on Developer column
    if test:
        te = pickle.load(open('encoders/dev_te.pkl', 'rb'))
    else:
        te = TargetEncoder(cols=['Developer']).fit(_df[['Developer']], _df['Average User Rating'])
        pickle.dump(te, open('encoders/dev_te.pkl', 'wb'))

    _df['dev_avg'] = te.transform(_df[['Developer']])

    return _df


### Frequency encoding approach

In [ ]:
def dev_preprocess_freq_enc(_df, test):
    # Convert to string
    _df['Developer'] = _df['Developer'].astype(str)
    _df['Developer'] = _df['Developer'].str.replace("'", "").str.strip('[]')
    
    if not test:
        ce = CountEncoder(cols=['Developer']).fit(_df[['Developer']])
        pickle.dump(ce, open('encoders/dev_ce.pkl', 'wb'))
    else:
        ce = pickle.load(open('encoders/dev_ce.pkl', 'rb'))
    
    _df['dev_freq'] = ce.transform(_df[['Developer']])['Developer']
        
    return _df


## Genres preprocessing

### 1. NLP approach (Bag of Words)

In [ ]:
def genres_preprocess_bow(_df, test=False):
    # Convert the genres column to a list of strings
    _df['Genres'] = _df['Genres'].astype(str)
    _df['Genres'] = _df['Genres'].str.strip('[]').str.replace("'", "").str.split(", ")

    # drop Games, Strategy, Entertainment from the Genres column
    _df['Genres'] = _df['Genres'].apply(lambda x: [genre for genre in x if genre not in ['Games', 'Strategy', 'Entertainment']])

    # Join the list of genres into a single string
    genres = _df['Genres'].apply(lambda x: ' '.join(x))

    # Create a count Vectorizer and fit it to the genres
    count_vec = CountVectorizer()
    bow_genres = count_vec.fit_transform(genres)
    
    # Save the count vectorizer to be used on the test set
    pickle.dump(count_vec, open('encoders/count_vec_genre.pkl', 'wb'))

    # Apply principal component analysis to reduce the dimensionality
    pca = PCA(n_components=10)
    pca_genres = pca.fit_transform(bow_genres.toarray())
    
    # Save the pca to be used on the test set
    pickle.dump(pca, open('encoders/genre_pca.pkl', 'wb'))

    # Add the PCA-transformed genres to the original dataframe
    for i in range(10):
        _df[f'genreN_{i}'] = pca_genres[:, i]

    return _df

### 2. Dummy variables approach

In [ ]:
def genres_preprocess_dummies(_df, test=False):
    # Convert the genres column to a list of strings
    _df['Genres'] = _df['Genres'].astype(str)
    _df['Genres'] = _df['Genres'].str.strip('[]').str.replace("'", "").str.split(", ")
    
    # drop Games, Strategy, Entertainment from the Genres column
    _df['Genres'] = _df['Genres'].apply(lambda x: [genre for genre in x if genre not in ['Games', 'Strategy', 'Entertainment']])
    
    if not test:
        
        # Replace genres with counts less than 100 with 'infrequent' as it would represent a very small percentage of the data (less than 2%)
        threshold = _df.shape[0] * 0.02
        
        other = _df['Genres'].explode().value_counts()[_df['Genres'].explode().value_counts() < threshold].index
        _df['Genres'] = _df['Genres'].apply(lambda x: [genre if genre not in other else 'infrequent' for genre in x])
        
        # Get dummy variables for the genres
        genres = pd.get_dummies(_df['Genres'].apply(pd.Series).stack(), prefix="genre", dummy_na=False).sum(level=0)
        
        # Save the genres dummies to be used on the test set
        genres.to_csv('encoders/genres.csv', index=False)
    
    else:
        # Load saved genres dummy variables
        saved_dummies = pd.read_csv('encoders/genres.csv')

        # Get the genres that are not in the saved dummy variables
        other = [genre for genre in _df['Genres'].explode().unique() if genre not in saved_dummies.columns]

        # Replace the genres that are not in the saved dummy variables with 'infrequent'
        _df['Genres'] = _df['Genres'].apply(lambda x: ['infrequent' if genre in other else genre for genre in x])

        # Preprocess test data using the saved dummy variables
        genres = pd.get_dummies(_df['Genres'].apply(pd.Series).stack(), prefix="genre", dummy_na=False).sum(level=0)
        genres = genres.reindex(columns=saved_dummies.columns, fill_value=0)
        
        # Fill the dummy columns with 0 if nan
        genres = genres.fillna(0)

    
    # Add the dummy variables to the original dataframe
    _df = pd.concat([_df, genres], axis=1)
    
    # Fill the NaN values with 0
    genre_cols = [col for col in _df.columns if col.startswith('genre')] # get all columns with prefix 'genre'
    _df[genre_cols] = _df[genre_cols].fillna(0) # fill the NaN values with 0
    
    return _df


## Languages preprocessing

### 1. NLP approach

In [ ]:
def lang_preprocessing_bow(_df):
    # Convert the langs column to a list of strings
    _df['Languages'] = _df['Languages'].astype(str)
    _df['Languages'] = _df['Languages'].str.strip('[]').str.replace("'", "").str.split(", ")
    
    # drop English from the Languages column
    _df['Languages'] = _df['Languages'].apply(lambda x: [lang for lang in x if lang not in ['EN']])
    
    # Join the list of langs into a single string
    languages = _df['Languages'].apply(lambda x: ' '.join(x))
    
    # Create a count Vectorizer and fit it to the langs
    count_vec = CountVectorizer()
    bow_languages = count_vec.fit_transform(languages)
    
    # Save the count vectorizer to be used on the test set
    pickle.dump(count_vec, open('encoders/count_vec_lang.pkl', 'wb'))
    
    # Apply principal component analysis to reduce the dimensionality
    pca = PCA(n_components=10)
    pca_languages = pca.fit_transform(bow_languages.toarray())
    
    # Save the pca to be used on the test set
    pickle.dump(pca, open('encoders/pca_lang.pkl', 'wb'))
    
    # Add the PCA-transformed langs to the original dataframe
    for i in range(len(pca_languages[0])):
        _df[f'lang_{i}'] = pca_languages[:, i]
        
    return _df


### 2. Dummy variables approach

In [ ]:
def langs_preprocess_dummies(_df, test = False):
    # Convert the langs column to a list of strings
    _df['Languages'] = _df['Languages'].astype(str)
    _df['Languages'] = _df['Languages'].str.strip('[]').str.replace("'", "").str.split(", ")
    
    # Create a column with the number of languages supported
    _df['langs_count'] = _df['Languages'].apply(lambda x: len(x)) 
    
    # drop English from the Languages column
    _df['Languages'] = _df['Languages'].apply(lambda x: [lang for lang in x if lang not in ['EN']])
    
    if not test:
        # Replace languages with counts less than 100 with 'infrequent' as it would represent a very small percentage of the data (less than 10%)
        threshold = _df.shape[0] * 0.1
        
        # Replace langs with counts less than 500 with 'infrequent_langs' as it would represent a very small percentage of the data (less than 10%)
        other = _df['Languages'].explode().value_counts()[_df['Languages'].explode().value_counts() < threshold].index
        _df['Languages'] = _df['Languages'].apply(lambda x: [lang if lang not in other else 'infrequent' for lang in x])

        # Get dummy variables for the langs
        langs = pd.get_dummies(_df['Languages'].apply(pd.Series).stack(), prefix='lang', dummy_na=False).sum(level=0)

        langs.to_csv('encoders/langs.csv', index=False)
    else:
        saved_dummies = pd.read_csv('encoders/langs.csv')

        # Get the languages that are not in the saved dummy variables
        other = [lang for lang in _df['Languages'].explode().unique() if lang not in saved_dummies.columns]

        # Replace the languages that are not in the saved dummy variables with 'infrequent'
        _df['Languages'] = _df['Languages'].apply(lambda x: ['infrequent' if lang in other else lang for lang in x])

        # Preprocess test data using the saved dummy variables
        langs = pd.get_dummies(_df['Languages'].apply(pd.Series).stack(), prefix="lang", dummy_na=False).sum(level=0)
        langs = langs.reindex(columns=saved_dummies.columns, fill_value=0)

        # Fill the dummy columns with 0 if nan
        langs = langs.fillna(0)

    # Add the dummy variables to the original dataframe
    _df = pd.concat([_df, langs], axis=1)

    # Fill NaN with 0
    lang_cols = [col for col in _df.columns if col.startswith('lang')] # get all columns with prefix 'lang'
    _df[lang_cols] = _df[lang_cols].fillna(0) # fill NaN with 0 for selected columns
    
    return _df
    
    

## In-app Purchases preprocessing

In [ ]:
# Free apps might skew the in-app purchases column,
# so we might split the dataset into free and paid apps

def purchases_preprocess(_df):
    # Convert the In-app Purchases column to a list of floats
    _df['In-app Purchases'] = _df['In-app Purchases'].astype(str)
    _df['In-app Purchases'] = _df['In-app Purchases'].str.strip('[]').str.replace("'", "").str.split(", ")

    # Convert to float  
    _df['In-app Purchases'] = _df['In-app Purchases'].apply(lambda x: [float(i) for i in x])

    # Get the number of in-app purchases
    _df['purchases_count'] = _df['In-app Purchases'].apply(lambda x: len(x))

    # Get the lowest, highest and average purchase
    _df['lowest_purchase'] = _df['In-app Purchases'].apply(lambda x: min(x) if len(x) > 0 else 0)
    _df['highest_purchase'] = _df['In-app Purchases'].apply(lambda x: max(x) if len(x) > 0 else 0)
    _df['average_purchase'] = _df['In-app Purchases'].apply(lambda x: np.mean(x) if len(x) > 0 else 0)

    _df['lowest_purchase'] = _df['lowest_purchase'].fillna(0)
    _df['highest_purchase'] = _df['highest_purchase'].fillna(0)
    _df['average_purchase'] = _df['average_purchase'].fillna(0)
    
    return _df

## Age Rating & Price preprocessing

In [ ]:
def age_preprocess(_df, test=False, df_old=None):
    # Convert to string
    _df['Age Rating'] = _df['Age Rating'].astype(str)

    # Remove the + sign
    _df['Age Rating'] = _df['Age Rating'].str.replace('+', '')

    # Convert to int
    _df['Age Rating'] = _df['Age Rating'].astype(float)
    
    if test:
        _df['Age Rating'] = _df['Age Rating'].fillna(df_old['Age Rating'].median())
    else:
        _df['Age Rating'] = _df['Age Rating'].fillna(_df['Age Rating'].median())
    
    return _df

In [ ]:
def price_preprocess(_df):
    # Convert to float
    _df['Price'] = _df['Price'].astype(float)

    # fill the missing values with 0 (free)
    _df['Price'] = _df['Price'].fillna(0)
    
    return _df

## NLP preprocessing

In [ ]:
def preprocess_nlp(_df, col):
    # Convert to string
    _df[col] = _df[col].astype(str)

    # Remove URLs and email addresses
    _df[col] = _df[col].apply(lambda x: re.sub(r'http\S+|www.\S+|\S+@\S+', '', x))

    # Remove the punctuation, numbers, and convert to lowercase
    _df[col] = _df[col].apply(lambda x: " ".join(re.findall(r'\w+', x.lower())))

    # Remove the stopwords
    _df[col] = _df[col].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))

    # Stemming
    st = nltk.PorterStemmer()
    _df[col] = _df[col].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

    # Lemmatization
    lem = nltk.WordNetLemmatizer()
    _df[col] = _df[col].apply(lambda x: " ".join([lem.lemmatize(word) for word in x.split()]))

    # Remove the frequent and rare words
    freq = pd.Series(' '.join(_df[col]).split()).value_counts()
    common_freq = list(freq[:10].index)
    rare_freq = list(freq[-10:].index)
    _df[col] = _df[col].apply(lambda x: " ".join(x for x in x.split() if x not in common_freq+rare_freq))

    # Remove the whitespaces
    _df[col] = _df[col].apply(lambda x: " ".join(x.strip() for x in x.split()))

    # Replace NaN values with empty string
    _df[col] = _df[col].fillna('')

    # Convert text data to bag-of-words representation
    vectorizer = CountVectorizer()
    BoW = vectorizer.fit_transform(_df[col])

    # Apply principal component analysis to reduce the dimensionality
    pca_ = PCA(n_components=2)
    pca_col = pca_.fit_transform(BoW.toarray())
    
    # Save the vectorizer and pca for later use with the test data
    pickle.dump(vectorizer, open(f'encoders/vectorizer_{col}.pkl', 'wb'))
    pickle.dump(pca_, open(f'encoders/pca_{col}.pkl', 'wb'))

    # Add the PCA-transformed col to the original dataframe
    for feat in range(len(pca_col[0])):
        _df[f'{col}_PCA_{feat}'] = pca_col[:, feat]
        
    return _df



## Description, Name & Subtitle preprocessing

In [ ]:

def compute_excitement_score(text, _sia):
    
    # compute the polarity scores for the given text
    scores = _sia.polarity_scores(text)
    
    # compute the excitement score as the sum of the positive and negative polarity scores
    excitement_score = scores['pos'] + abs(scores['neg'])
    
    return excitement_score

# define a function to compute an attractive score from a given text
def compute_attractive_score(text, tokenizer):
    # define a list of keywords that might make a game attractive to users
    attractive_keywords = ['graphics', 'gameplay', 'storyline', 'characters']
    
    # tokenize the text into words and count how many attractive keywords appear
    words = tokenizer(text.lower())
    
    num_attractive_keywords = len([word for word in words if word in attractive_keywords])
    
    # compute the attractive score as the ratio of attractive keywords to total words
    attractive_score = num_attractive_keywords / len(words) if len(words) > 0 else 0
    
    return attractive_score

def desc_preprocess(_df, test=False):
    _df['Description'] = _df['Description'].astype(str)
    
    if not test:        
        # load the Sentiment Intensity Analyzer model from NLTK
        sia_desc = SentimentIntensityAnalyzer()
        pickle.dump(sia_desc, open('encoders/sia_desc.pkl', 'wb'))

        tokenizer = nltk.word_tokenize
        pickle.dump(nltk.word_tokenize, open('encoders/desc_tokenizer.pkl', 'wb'))
        
    else:
        sia_desc = pickle.load(open('encoders/sia_desc.pkl', 'rb'))
        tokenizer = pickle.load(open('encoders/desc_tokenizer.pkl', 'rb'))
    
    _df['excitement_score'] = _df['Description'].apply(lambda x : compute_excitement_score(x, sia_desc))
    _df['attractive_score'] = _df['Description'].apply(lambda x: compute_attractive_score(x, tokenizer))
    
    return _df

def name_preprocess(_df, test=False):
    _df['Name'] = _df['Name'].astype(str)
    
    if not test:
        sia_name = SentimentIntensityAnalyzer()
        pickle.dump(sia_name, open('encoders/sia_name.pkl', 'wb'))
    else:
        sia_name = pickle.load(open('encoders/sia_name.pkl', 'rb'))
    
    _df['name_sia'] = _df['Name'].apply(lambda x : compute_excitement_score(x, sia_name))
    
    return _df

def sub_preprocess(_df, test=False):
    _df['Subtitle'] = _df['Subtitle'].astype(str)
    
    if not test:
        sia_sub = SentimentIntensityAnalyzer()
        pickle.dump(sia_sub, open('encoders/sia_sub.pkl', 'wb'))
    else:
        sia_sub = pickle.load(open('encoders/sia_sub.pkl', 'rb'))
    
    _df['sub_sia'] = _df['Subtitle'].apply(lambda x : compute_excitement_score(x, sia_sub))
    
    return _df

## Icon preprocessing

In [ ]:
def preprocess_icon(img_path):
    
        # Load the game icon image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256))

    # Extract color features using color histograms
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    color_features = []
    for i in range(3):
        hist = cv2.calcHist([img_lab], [i], None, [256], [0, 256])
        color_features.append(hist.ravel())
    color_features = np.concatenate(color_features)

    # Extract shape features using local binary patterns
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, 8, 1, method='uniform')
    hist_lbp, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    edge_features = hist_lbp.astype(float)

    # Combine the color and shape features into a single feature vector
    feature_vector = np.concatenate((color_features, edge_features))

    # Normalize the feature vector to have unit length
    normalized_feature_vector = feature_vector / np.linalg.norm(feature_vector)
    
    return normalized_feature_vector


In [ ]:
def icons_preprocess(_df, test=False):

    # Create a list to store the feature vectors
    icon_features = []

    _df['Icon URL'] = _df['Icon URL'].astype(str)

    # Iterate over the images and extract the features
    for i, row in tqdm(_df.iterrows(), total=_df.shape[0]):
        feature_vec = preprocess_icon(row['Icon URL'])
        icon_features.append((row['Icon URL'], feature_vec))
        
    if not test:
        # Apply PCA to reduce the number of features
        pca = PCA(n_components=4)
        pca.fit([f[1] for f in icon_features])

        # Save the pca instance for later use
        pickle.dump(pca, open('encoders/icon_pca.pkl', 'wb'))
    
    else:
        pca = pickle.load(open('encoders/icon_pca.pkl', 'rb'))
    
    reduced_features = pca.transform([f[1] for f in icon_features])

    # Convert the reduced features to a dataframe
    icon_features_df = pd.DataFrame({'Icon URL': [f[0] for f in icon_features],
                                        'Icon1': reduced_features[:,0],
                                        'Icon2': reduced_features[:,1],
                                        'Icon3': reduced_features[:,2],
                                        'Icon4': reduced_features[:,3]})
    
    # Merge the icon features with the original dataframe on the icon URL
    _df = _df.merge(icon_features_df, on='Icon URL', how='left')
    
    return _df

## Pipeline

In [ ]:
def preprocess_pipeline(_df, test=False):
    _df = _df.drop(['Primary Genre', 'ID', 'URL'], axis=1)
    
    if test:
        df_old = pd.read_csv('preprocessed_data.csv') 
    else:
        df_old = None
          
    _df = reviews_preprocess(_df, test)
    _df = date_preprocess(_df, test, df_old)
    
    _df = dev_preprocess_freq_enc(_df, test)
    _df = dev_preprocess_target_enc(_df, test)
    
    _df = genres_preprocess_dummies(_df, test)
    _df = langs_preprocess_dummies(_df, test)
    
    _df = purchases_preprocess(_df)
    
    _df = age_preprocess(_df, test, df_old)
    _df = price_preprocess(_df)
    
    _df = name_preprocess(_df, test)
    _df = sub_preprocess(_df, test)
    _df = desc_preprocess(_df, test)
    
    _df = icons_preprocess(_df, test)
    
    _df = _df.drop(['Developer',
              'Genres',
              'Languages',
              'Reviews',
              'In-app Purchases',
              'Description',
              'Subtitle',
              'Name', 
              'Icon URL'], axis=1)
    
    if not test:
        _df.to_csv('preprocessed_data.csv', index=False)
        
    # Columns needed for the testing phase but nor for the prediction model anymore
    _df = _df.drop(['Original Release Date',
              'Current Version Release Date'], axis=1)
    
    return _df

In [ ]:
df, df_test = train_test_split(df_origin, test_size=0.2, random_state=42)

df = preprocess_pipeline(df)
df_test = preprocess_pipeline(df_test, test=True)

df_x, df_y = df.drop('Average User Rating', axis=1), df['Average User Rating']
df_test_x, df_test_y = df_test.drop('Average User Rating', axis=1), df_test['Average User Rating']

## Scaling and Model Training

In [ ]:
def plot_scores(mse_train, r2_train, mse_test, r2_test):
    # Calculate the differences between the training and testing scores
    mse_diff =  abs(mse_train - mse_test)
    r2_diff = abs(r2_train - r2_test)

    # Determine whether each score is better or worse than the other
    mse_color = 'green' if mse_diff < 0.1 else 'red'
    r2_color = 'green' if r2_diff < 0.1 else 'red'
    
    # Create a bar plot of the scores with colors based on the value
    fig = go.Figure(data=[
        go.Bar(name='MSE Train', x=['MSE'], y=[mse_train], marker_color='navy'),
        go.Bar(name='MSE Test', x=['MSE'], y=[mse_test], marker_color='blue'),
        go.Bar(name='MSE Difference', x=['MSE'], y=[mse_diff], marker_color=mse_color),
        go.Bar(name='R2 Train', x=['R2'], y=[r2_train], marker_color='purple'),
        go.Bar(name='R2 Test', x=['R2'], y=[r2_test], marker_color='pink'),
        go.Bar(name='R2 Difference', x=['R2'], y=[r2_diff], marker_color=r2_color)
    ])

    # Add labels and title
    fig.update_layout(title='Model Performance', xaxis_title='Score Type', yaxis_title='Score')

    # Show the plot
    fig.show()

### Feature Scaling

In [ ]:
# Scale the features

def scale_data_std(_df, test=False):
    cols = _df.columns
    
    if not test:
        scaler = StandardScaler()
        _df = scaler.fit_transform(_df)
        
        # Save the scaler
        pickle.dump(scaler, open('scalers/std_scaler.pkl', 'wb'))
    else:
        scaler = pickle.load(open('scalers/std_scaler.pkl', 'rb'))
        _df = scaler.transform(_df)
    
    _df = pd.DataFrame(_df, columns=cols)
    return _df

def scale_data_minmax(_df, test=False):
    cols = _df.columns
    
    if not test:
        scaler = MinMaxScaler()
        _df = scaler.fit_transform(_df)
        
        # Save the scaler
        pickle.dump(scaler, open('scalers/minmax_scaler.pkl', 'wb'))
    
    else:
        scaler = pickle.load(open('scalers/minmax_scaler.pkl', 'rb'))
        _df = scaler.transform(_df)
    
    _df = pd.DataFrame(_df, columns=cols)
    return _df


In [ ]:
# plot correlation matrix between features and target

corr = df_x.corrwith(df_y)
corr = corr.sort_values(ascending=False)

plt.figure(figsize=(10, 10))
sns.barplot(x=corr.values, y=corr.index)
# plt.show()

### Feature Selection

In [ ]:
def select_features(_df_x, _df_y, num_features=8, test=False):
    if not test:
        selector = SelectKBest(f_regression, k=num_features)
        selector.fit(_df_x, _df_y)
        
        # Save the selector
        pickle.dump(selector, open('encoders/selector.pkl', 'wb'))
        
    else:
        selector = pickle.load(open('encoders/selector.pkl', 'rb'))
        
    _df_x = selector.transform(_df_x)
    
    return _df_x

In [ ]:
# Scale the features
df_x_scaled = scale_data_minmax(df_x)
df_test_x_scaled = scale_data_minmax(df_test_x, test=True)

# Select the features
df_x_select = select_features(df_x_scaled, df_y, 12)
df_test_x_select = select_features(df_test_x_scaled, df_test_y, test=True)

In [ ]:
selector = pickle.load(open('encoders/selector.pkl', 'rb'))
# Print the selected features
for i in range(len(selector.get_support())): 
    if selector.get_support()[i]:
        print(df_x.columns[i])

### Linear Regression

In [ ]:
# Create a linear regression model
model = LinearRegression()

# Train the model
model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, model.predict(df_x_select))
r2_train = r2_score(df_y, model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)

# Save the model
pickle.dump(model, open('models/LR_model.pkl', 'wb'))

### Ridge Regression

In [ ]:
# Create a ridge regression model
model = Ridge(alpha=20)

# Train the model
model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, model.predict(df_x_select))
r2_train = r2_score(df_y, model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)

# Save the model
pickle.dump(model, open('models/Ridge_model.pkl', 'wb'))


### Lasso Regression

In [ ]:
# Create a lasso regression model
model = Lasso(alpha=0.001)

# Train the model
model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, model.predict(df_x_select))
r2_train = r2_score(df_y, model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)

# Save the model
pickle.dump(model, open('models/Lasso_model.pkl', 'wb'))


### Elastic Net Regression

In [ ]:
# Create an elastic net regression model
model = ElasticNet(alpha=0.001)

# Train the model
model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, model.predict(df_x_select))
r2_train = r2_score(df_y, model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)

# Save the model
pickle.dump(model, open('models/ElasticNet_model.pkl', 'wb'))


### Polynomial Regression

In [ ]:
# Create a polynomial regression model
model = LinearRegression()

# Generate the polynomial features
poly = PolynomialFeatures(degree=2)
df_x_poly = poly.fit_transform(df_x_select)
df_test_x_poly = poly.transform(df_test_x_select)

# Save the polynomial features
pickle.dump(poly, open('encoders/poly.pkl', 'wb'))

# Train the model
model.fit(df_x_poly, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, model.predict(df_x_poly))
r2_train = r2_score(df_y, model.predict(df_x_poly))
mse_test = mean_squared_error(df_test_y, model.predict(df_test_x_poly))
r2_test = r2_score(df_test_y, model.predict(df_test_x_poly))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)
# Save the model
pickle.dump(model, open('models/Polynomial_model.pkl', 'wb'))


### XGBoost

In [ ]:
# Create an XGBoost model

import xgboost as xgb

model = xgb.XGBRegressor(subsample= 0.8,
                         reg_lambda= 0.1,
                         reg_alpha= 0.1,
                         n_estimators=500,
                         min_child_weight=1,
                         max_depth= 4,
                         learning_rate= 0.01,
                         gamma =0.1,
                         colsample_bytree= 0.8)

# Train the model
model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, model.predict(df_x_select))
r2_train = r2_score(df_y, model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)

# Save the model
pickle.dump(model, open('models/XGBoost_model.pkl', 'wb'))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define the XGBoost regressor
xgb_model = xgb.XGBRegressor()

# Define the hyperparameter grid for RandomizedSearchCV
params = {
    'learning_rate': [0.01],
    'max_depth': [3, 4],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'n_estimators': [500, 1000],
    'min_child_weight': [1],
    'gamma': [0.1],
    'reg_alpha': [0.1],
    'reg_lambda': [0.1]
}

# Define the RandomizedSearchCV object
rs_cv = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=params,
    n_iter=10,
    cv=5,
    random_state=42
)

# Fit the RandomizedSearchCV object to the data
rs_cv.fit(df_x_select, df_y)

# Print the best hyperparameters
print(rs_cv.best_params_)

# Use the best hyperparameters to train the final model
final_xgb_model = xgb.XGBRegressor(**rs_cv.best_params_)
final_xgb_model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, final_xgb_model.predict(df_x_select))
r2_train = r2_score(df_y, final_xgb_model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, final_xgb_model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, final_xgb_model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)


### Gradient Boosting

In [ ]:
# Create a Gradient Boosting model

from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(learning_rate=0.01, 
                                  max_depth=5, 
                                  max_features='sqrt', 
                                  min_samples_leaf=1, 
                                  min_samples_split=2, 
                                  n_estimators=500, 
                                  subsample=0.8)

# Train the model
model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, model.predict(df_x_select))
r2_train = r2_score(df_y, model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)

# Save the model
pickle.dump(model, open('models/GradientBoosting_model.pkl', 'wb'))

In [ ]:
# Gradient Boosting with RandomizedSearchCV on GPU

from sklearn.model_selection import RandomizedSearchCV

# Define the Gradient Boosting regressor
gb_model = GradientBoostingRegressor()

# Define the hyperparameter grid for RandomizedSearchCV
params = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [4, 5],
    'subsample': [0.8],
    'n_estimators': [500, 1000],
    'min_samples_split': [2, 3],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Define the RandomizedSearchCV object
rs_cv = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=params,
    n_iter=10,
    cv=5,
    random_state=42
)

# Fit the RandomizedSearchCV object to the data
rs_cv.fit(df_x_select, df_y)
 
# Print the best hyperparameters
print(rs_cv.best_params_)

# Use the best hyperparameters to train the final model
final_gb_model = GradientBoostingRegressor(**rs_cv.best_params_)
final_gb_model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, final_gb_model.predict(df_x_select))
r2_train = r2_score(df_y, final_gb_model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, final_gb_model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, final_gb_model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)
 
# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)

### Random Forest Regression

In [ ]:
# Create a Random Forest Regressor model

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=5, 
                              n_estimators=500, 
                              n_jobs=-1)

# Train the model
model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, model.predict(df_x_select))
r2_train = r2_score(df_y, model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)
# Save the model
pickle.dump(model, open('models/RandomForest_model.pkl', 'wb'))

In [ ]:
## Random Forest with GridSearchCV on GPU
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV

# Define the Random Forest regressor
rf_model = RandomForestRegressor()

# Define the hyperparameter grid for GridSearchCV
params = {
    'max_depth': [4, 5],
    'n_estimators': [500, 1000],
    'max_features': ['auto', 'sqrt', 'log2'],
    'random_state': [42],
    'verbose': [1],
    'warm_start': [True],
    'bootstrap': [True],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 3],
    'criterion': ['squared_error'],
    'oob_score': [True]
}

# Define the GridSearchCV object
gs_cv = GridSearchCV(
    estimator=rf_model,
    param_grid=params,
    cv=5,
    n_jobs=6,
    scoring='neg_mean_squared_error',
    return_train_score=True,
    refit=True,
)

# Fit the GridSearchCV object to the data
gs_cv.fit(df_x_select, df_y)

# Print the best hyperparameters
print(gs_cv.best_params_)

# Use the best hyperparameters to train the final model
final_rf_model = RandomForestRegressor(**gs_cv.best_params_)

final_rf_model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, final_rf_model.predict(df_x_select))
r2_train = r2_score(df_y, final_rf_model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, final_rf_model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, final_rf_model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)

In [ ]:

# Create CatBoost Regressor model

from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=250,
                            learning_rate=0.05,
                            depth=3,
                            l2_leaf_reg=1,
                            border_count=32,
                            bagging_temperature=1,
                            fold_permutation_block=1,
                            boosting_type='Plain',
                            random_seed=42,
                            subsample=1.0,
                            colsample_bylevel=0.7,
                            early_stopping_rounds=5,
                            loss_function='RMSE', eval_metric='RMSE',
                          verbose=50)

# Train the model
model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, model.predict(df_x_select))
r2_train = r2_score(df_y, model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)

# Save the model
pickle.dump(model, open('models/CatBoost_model.pkl', 'wb'))

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

# Define hyperparameters to tune
param_grid = {
    'iterations': [250],
    'learning_rate': [0.05],
    'depth': [3],
    'l2_leaf_reg': [1],
    'border_count': [32],
    'bagging_temperature': [1],
    'fold_permutation_block': [1],
    'boosting_type': ['Plain'],
    'random_seed': [42],
    'subsample': [1.0],
    'colsample_bylevel': [0.7],
    'early_stopping_rounds': [5],
}

# Create a CatBoost regressor
clf = CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE', verbose=250)

# Grid Search with cross-validation
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(df_x_select, df_y)

# Print best hyperparameters and corresponding score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# Train final model with best hyperparameters on entire dataset
best_clf = CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE', logging_level='Silent', **grid_search.best_params_)
best_clf.fit(df_x_select, df_y, eval_set=(df_test_x_select, df_test_y))

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, best_clf.predict(df_x_select))
r2_train = r2_score(df_y, best_clf.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, best_clf.predict(df_test_x_select))
r2_test = r2_score(df_test_y, best_clf.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)


In [ ]:
# print catboost weights with the feature names and biases
print(model.get_feature_importance(prettified=True)) 

In [ ]:
# Create AdaBoost Regressor model

from sklearn.ensemble import AdaBoostRegressor

model = AdaBoostRegressor(n_estimators=100, learning_rate=0.01)

# Train the model
model.fit(df_x_select, df_y)

# Get the training MSE, R2 score, Testing MSE, and R2 score
mse_train = mean_squared_error(df_y, model.predict(df_x_select))
r2_train = r2_score(df_y, model.predict(df_x_select))
mse_test = mean_squared_error(df_test_y, model.predict(df_test_x_select))
r2_test = r2_score(df_test_y, model.predict(df_test_x_select))

# Print the training MSE and R2 score
print('Training MSE: ', mse_train)
print('Training R2: ', r2_train)

# Print the testing MSE and R2 score
print('Testing MSE: ', mse_test)
print('Testing R2: ', r2_test)

# Display the scores
plot_scores(mse_train, r2_train, mse_test, r2_test)
# Save the model
pickle.dump(model, open('models/AdaBoost_model.pkl', 'wb'))

## Done